In [32]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tqdm import tnrange

In [14]:
rsvps = pd.read_csv("data/rsvp4.csv")
rsvps.head()

,id,event_id,rsvp_status
0,1490062834615246,1003506093022823,attending
1,1522214584713449,1003506093022823,attending
2,1564556903780155,1003506093022823,attending
3,393778070776133,1003506093022823,attending
4,498288810314601,1003506093022823,attending


In [15]:
rsvps.shape

(19771, 3)

In [19]:
rsvps.rsvp_status.value_counts()

attending    12078
unsure        7693
Name: rsvp_status, dtype: int64

In [85]:
# rsvps["prob"] = rsvps["rsvp_status"].map({"attending": 1., "unsure": 0.5})
## will be simpler:
rsvps["prob"] = rsvps["rsvp_status"].map({"attending": 1., "unsure": 1.})

In [86]:
X = rsvps \
  .pivot(index="id", columns="event_id", values="prob") \
  .fillna(0.)

In [87]:
X.shape

(2568, 351)

In [88]:
len(rsvps) / (X.shape[0] * X.shape[1])

0.0219344374328798

In [89]:
probs_true = X.values.astype(np.float32)

In [97]:
n_users, n_events = probs_true.shape
n_features = 5
n_users = n_users

n_attending = probs_true.sum()
n_skipping = (1. - probs_true).sum()

print(probs_true.shape)

(2568, 351)


In [108]:
users = tf.Variable(tf.random_uniform([n_users, n_features], -1.0, 1.0))
events = tf.Variable(tf.random_uniform([n_features, n_events], -1.0, 1.0))

logits = tf.matmul(users, events)
probs_pred = 1./(1. + tf.exp(logits))
loss = - tf.reduce_mean(probs_true * tf.log(probs_pred)) - tf.reduce_mean((1. - probs_true) * tf.log(1. - probs_pred))
mean_attedning = tf.reduce_sum(probs_true * probs_pred) / n_attending
mean_skipping = tf.reduce_sum((1. - probs_true) * probs_pred) / n_skipping

In [109]:
# optimizer = tf.train.GradientDescentOptimizer(0.5)
optimizer = tf.train.AdamOptimizer(0.05)
train = optimizer.minimize(loss)

In [110]:
# Before starting, initialize the variables.  We will 'run' this first.
init = tf.initialize_all_variables()  # old version
# init = tf.global_variables_initializer()

# Launch the graph.
sess = tf.Session()
sess.run(init)

# Fit the line.
for step in tnrange(2000):
    sess.run(train)
    if step % 20 == 0:
        print(step, sess.run(loss), sess.run(mean_attedning), sess.run(mean_skipping))

0 0.741995 0.500186 0.499872
20 0.137381 0.0928146 0.0792909
40 0.0883536 0.0936755 0.0145198
60 0.0807772 0.103608 0.0195471



Exception in thread Thread-8:
Traceback (most recent call last):
  File "/Users/pmigdal/anaconda3/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/Users/pmigdal/anaconda3/lib/python3.5/site-packages/tqdm/_tqdm.py", line 102, in run
    for instance in self.tqdm_cls._instances:
  File "/Users/pmigdal/anaconda3/lib/python3.5/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



80 0.0789423 0.106819 0.0203227
100 0.0777495 0.109309 0.0198725
120 0.0760569 0.120954 0.0198424
140 0.0735741 0.141431 0.0195153
160 0.0706505 0.16814 0.019002
180 0.0679356 0.192529 0.0184446
200 0.0657064 0.211804 0.0179847
220 0.0639173 0.227509 0.0175944
240 0.0624437 0.240897 0.0172507
260 0.0611945 0.252577 0.0169504
280 0.0601466 0.262527 0.0166913
300 0.0593074 0.270338 0.0164822
320 0.0586556 0.276072 0.016326
340 0.0581465 0.280256 0.0162121
360 0.0577404 0.28342 0.0161263
380 nan nan nan
400 nan nan nan
420 nan nan nan
440 nan nan nan
460 nan nan nan
480 nan nan nan
500 nan nan nan
520 nan nan nan



KeyboardInterrupt: 

In [105]:
sess.run(events)

array([[ nan,  nan,  nan, ...,  nan,  nan,  nan],
       [ nan,  nan,  nan, ...,  nan,  nan,  nan],
       [ nan,  nan,  nan, ...,  nan,  nan,  nan],
       [ nan,  nan,  nan, ...,  nan,  nan,  nan],
       [ nan,  nan,  nan, ...,  nan,  nan,  nan]], dtype=float32)

836/|/ 42%|| 836/2000 [00:40<00:55, 20.89it/s]